In [1]:
import numpy as np
from pyspark import SparkContext

sc = SparkContext("local", "pyspark")

Implementing Matrix Multiplication using Spark RDD's **transformation** and **action** operations

Initializing two random matricies of dimensions (3, 3) and (3, 5)

In [2]:
A = np.random.rand(3, 3)
B = np.random.rand(3, 5)

In [3]:
A

array([[ 0.32757648,  0.24234605,  0.27544155],
       [ 0.3635764 ,  0.26453351,  0.57541669],
       [ 0.99410419,  0.76877575,  0.0465294 ]])

In [4]:
B

array([[ 0.57506109,  0.2772601 ,  0.21213679,  0.65079861,  0.68497444],
       [ 0.34860906,  0.99456752,  0.04617413,  0.22087513,  0.80705158],
       [ 0.84801544,  0.20709871,  0.16276561,  0.26294889,  0.22046303]])

Their product is

In [5]:
A.dot(B)

array([[ 0.50643921,  0.38889699,  0.12551355,  0.33914159,  0.48069196],
       [ 0.78925966,  0.48306972,  0.18300058,  0.44634907,  0.58939084],
       [ 0.87913047,  1.049861  ,  0.25395701,  0.82899992,  1.31163566]])

Turn them into Spark RDD:

In [6]:
rddA = sc.parallelize(list(enumerate(A.T)))
rddA = rddA.flatMapValues(lambda x: list(enumerate(x)))

rddB = sc.parallelize(list(enumerate(B)))
rddB = rddB.flatMapValues(lambda x: list(enumerate(x)))

This is what they look like:

In [7]:
rddA.collect()

[(0, (0, 0.32757648142963669)),
 (0, (1, 0.36357640461103735)),
 (0, (2, 0.99410418764654573)),
 (1, (0, 0.24234605339372484)),
 (1, (1, 0.26453351012894)),
 (1, (2, 0.7687757540988811)),
 (2, (0, 0.27544155171434204)),
 (2, (1, 0.57541668702403537)),
 (2, (2, 0.046529398094665275))]

In [8]:
rddB.collect()

[(0, (0, 0.57506108811795209)),
 (0, (1, 0.27726010273623725)),
 (0, (2, 0.21213678828319382)),
 (0, (3, 0.65079861025993913)),
 (0, (4, 0.68497443743774711)),
 (1, (0, 0.34860905609986237)),
 (1, (1, 0.99456751546238809)),
 (1, (2, 0.046174134233585185)),
 (1, (3, 0.22087512903438833)),
 (1, (4, 0.8070515829112388)),
 (2, (0, 0.84801544269731977)),
 (2, (1, 0.20709871229581445)),
 (2, (2, 0.16276560654208061)),
 (2, (3, 0.26294889424979384)),
 (2, (4, 0.22046302721617617))]

Join them on A's column index (or B's row index), and do dot-product on A's row vectors with B's column vectors:

In [9]:
C = rddA.join(rddB).map(lambda x: ((x[1][0][0], x[1][1][0]),
                                   x[1][0][1] * x[1][1][1])).reduceByKey(lambda x, y: x + y)

C.collect()

[((1, 3), 0.44634907361366755),
 ((0, 4), 0.48069196040086404),
 ((1, 1), 0.4830697221366137),
 ((2, 0), 0.87913047396319355),
 ((0, 0), 0.50643920619016469),
 ((2, 2), 0.25395701015441519),
 ((2, 4), 1.3116356578954811),
 ((0, 2), 0.12551355311965062),
 ((0, 1), 0.38889699177386955),
 ((1, 2), 0.18300058265361327),
 ((2, 1), 1.0498609993285575),
 ((2, 3), 0.82899992143815804),
 ((1, 4), 0.5893908360287331),
 ((0, 3), 0.33914158613629569),
 ((1, 0), 0.78925965670458154)]

Finally, clean up the result **C**:

In [10]:
C = C.map(lambda x: (x[0][0],(x[0][1], x[1]))).groupByKey()
C = C.mapValues(list).mapValues(lambda x: sorted(x, key=lambda y: y[0]))
C = C.mapValues(lambda x: zip(*x)[1])

C = np.array(C.sortByKey().map(lambda x: np.array(x[1])).collect())

In [11]:
C

array([[ 0.50643921,  0.38889699,  0.12551355,  0.33914159,  0.48069196],
       [ 0.78925966,  0.48306972,  0.18300058,  0.44634907,  0.58939084],
       [ 0.87913047,  1.049861  ,  0.25395701,  0.82899992,  1.31163566]])

which equals A.dot(B):

In [12]:
A.dot(B)

array([[ 0.50643921,  0.38889699,  0.12551355,  0.33914159,  0.48069196],
       [ 0.78925966,  0.48306972,  0.18300058,  0.44634907,  0.58939084],
       [ 0.87913047,  1.049861  ,  0.25395701,  0.82899992,  1.31163566]])